Though this was not needed since the data is artificially generated, it is a good practice to perform a check

# Adversarial Validation

**Adversarial validation is a technique applied to the data to help reduce overfitting. The idea behind this is to identify the degree of similarity between the training and test data by analysing feature distribution. To do this, we build a classifier to predict which data is from the training set and which is from the test set. This model will assign 1 for rows from the train set and 0 for rows of data from the test set. If there are differences between the distributions of train and test set then this classifier will be able to identify these. The better a model you can learn to distinguish them, the bigger the problem you have.**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
train_data = pd.read_csv('../input/tabular-playground-series-mar-2021/train.csv')

train_data.drop(['id', 'target'], axis = 1, inplace = True)

test_data = pd.read_csv('../input/tabular-playground-series-mar-2021/test.csv')

test_data.drop(['id'], axis = 1, inplace = True)

In [ ]:
train_data['istrain'] = 1

test_data['istrain'] = 0

combined_data = pd.concat([train_data, test_data], axis = 0)

In [ ]:
df_numeric = combined_data.select_dtypes(exclude=['object'])

df_obj = combined_data.select_dtypes(include=['object']).copy()
    
for c in df_obj:
    df_obj[c] = pd.factorize(df_obj[c])[0]
    
combined_data = pd.concat([df_numeric, df_obj], axis=1)

y = combined_data['istrain']

combined_data.drop('istrain', axis = 1, inplace = True)

In [ ]:
combined_data.head()

In [ ]:
skf = StratifiedShuffleSplit(n_splits = 5, random_state = 44,test_size =0.4)
xgb_params = {
        'learning_rate': 0.1, 'max_depth': 6,'subsample': 0.9,
        'colsample_bytree': 0.9,'objective': 'binary:logistic',
        'n_estimators':100, 'gamma':1,
        'min_child_weight':4,'use_label_encoder' :False
        }   
clf = xgb.XGBClassifier(**xgb_params, seed = 10)    

In [ ]:
scores = []

avg_loss = []

for train_index, test_index in skf.split(combined_data, y):
       
        x0, x1 = combined_data.iloc[train_index], combined_data.iloc[test_index]
        
        y0, y1 = y.iloc[train_index], y.iloc[test_index]        
        
        print(x0.shape)
        
        clf.fit(x0, y0, eval_set=[(x1, y1)],
               eval_metric=['logloss','auc'], verbose=True,early_stopping_rounds=50)
                
        prval = clf.predict_proba(x1)[:,1]
        
        roc = roc_auc_score(y1,prval)

        scores.append(roc)
        
        avg_loss.append(clf.best_score)

        print ('XGB Val OOF AUC=',roc)
    
print("Log Loss Stats {0:.8f},{1:.8f}".format(np.array(avg_loss).mean(), np.array(avg_loss).std()))

print('%.8f (%.8f)' % (np.array(scores).mean(), np.array(scores).std()))

**As we can see the AUC socre is close to 0.5,whic implies train and test data are almost indistiguishable. Both train and test set must come from similar distribution and normal validation techniques should work**